# Clean C11 Districts Excel file

We have a file containing all the census blocks and MCD codes for each NJ Congressional district. We don't have the mapping from those codes to the voter dataset that we've created. This is where we fix that. 

Plan: 
* drop 1 row at start and two rows at end
* for district 001 to 012, find "District xxx" and "District xxx Subtotal"; make everything between then district = xxx; drop the top and bottom row
* Look for "County",  make everything between xxx County and xxx County Subtal county= xxx; drop the top and bottom rows
* Get the name of each MCD
* Get the mapping from each non-MCD tract to the voters dataset (we have ward, zipcode and address: somewhere in there we can do this mapping). 
* Erm... that should be it.  Unless you want to mark the towns vs tracts?

In [118]:
import pandas as pd

df = pd.read_excel('../2017 Other data/NJCD_2011_PLAN_COMPONENTS_COUNTY_MCD_XLS.xls')
df = df[['Plan:', 'NJCD_2011_ADOPTED_DEC23']]

# Convert census blocks to MCD tag plus block id
df = df.fillna('')
df.rename(columns={'Plan:': 'Census Block', 
                   'NJCD_2011_ADOPTED_DEC23': 'Population'}, inplace=True)
df['Census Block'] = df['Census Block'].str.strip()
df['MCD'] = 'N'
df.loc[df['Census Block'].str.startswith('MCD'), ['MCD']] = 'Y'
df.loc[df['Census Block'].str.startswith('MCD'), ['Census Block']] = df['Census Block'].str[4:].str.strip()

# Add block names, district and county ids
df['Name'] = ''
df['District'] = ''
df['County'] = ''
df.head()

,Census Block,Population,MCD,Name,District,County
0,,Population,N,,,
1,District 001,,N,,,
2,Burlington NJ County,,N,,,
3,3400543740,19131,Y,,,
4,3400555800,7398,Y,,,


In [119]:
df[df['Census Block'].str.contains('Subtotal')]['Census Block'].unique()

array(['Burlington NJ County Subtotal', 'Camden NJ County Subtotal',
       '3401519180 Subtotal', 'Gloucester NJ County Subtotal',
       'District 001 Subtotal', '3402970320 Subtotal',
       'Ocean NJ County Subtotal', 'District 002 Subtotal',
       '3402959880 Subtotal', 'District 003 Subtotal',
       'Mercer NJ County Subtotal', '3402545990 Subtotal',
       'Monmouth NJ County Subtotal', 'District 004 Subtotal',
       '3400372360 Subtotal', 'Bergen NJ County Subtotal',
       'Passaic NJ County Subtotal', 'Sussex NJ County Subtotal',
       'Warren NJ County Subtotal', 'District 005 Subtotal',
       '3402354705 Subtotal', 'Middlesex NJ County Subtotal',
       'District 006 Subtotal', 'Essex NJ County Subtotal',
       'Morris NJ County Subtotal', 'Somerset NJ County Subtotal',
       '3403966060 Subtotal', '3403974480 Subtotal',
       'Union NJ County Subtotal', 'District 007 Subtotal',
       '3401351000 Subtotal', '3401703580 Subtotal', '3401736000 Subtotal',
       '3401

In [120]:
# Put district and county into their own columns
district = ''
county = ''
for row in df['Census Block'].iteritems():
    
    # Find district and county ids
    if row[1] in ['State totals', '']:
        continue
    if row[1].find('Subtotal') > -1:
        continue
    if row[1].startswith('District'):
        district = row[1][8:].strip()
        continue
    if row[1].endswith('NJ County'):
        county = row[1][:-9].strip()
        continue
    
    df.set_value(row[0], 'District', district)
    df.set_value(row[0], 'County', county)

df = df[df['District'] != '']
df.head()

,Census Block,Population,MCD,Name,District,County
3,3400543740,19131,Y,,001,Burlington
4,3400555800,7398,Y,,001,Burlington
7,3400702200,8819,Y,,001,Camden
8,3400702230,1023,Y,,001,Camden
9,3400703250,6983,Y,,001,Camden


In [121]:
# Add MCD names
header = ['Summary level code', 'FIPS state code', 
          'FIPS county code', 'FIPS MCD code', 'State name', 
          'County name', 'MCD name', 
          'resident pop estimate', 'resident pop MOE', 
          'working in MCD', 'working in MCD MOE', 
          'living in MCD', 'living in MCD MOE', 
          'daytime pop', 'daytime pop MOE',
          'commuting pop change', 'commuting pop change MOE',
          'commuting pop percent', 'commuting pop percent MOE',
          'live and work', 'live and work MOE',
          'live and work percent', 'live and work percent MOE',
          'work residence ratio', 'work residence ratio MOE']
mcdheadertypes = {}
for col in header:
    mcdheadertypes[col] = 'str'

dfmcd = pd.read_excel('../2017 Other data/MCD details/Table2.xls', skiprows=7, skipfooter=22,
                      dtypes = mcdheadertypes, header = None, names=header, index=None)
dfmcd = dfmcd.fillna('')
for col in ['FIPS state code', 'FIPS county code', 'FIPS MCD code']:
    dfmcd[col] = dfmcd[col].astype(str)
dfmcd['census block'] = dfmcd['FIPS state code'].str.zfill(2) + \
                        dfmcd['FIPS county code'].str.zfill(3) + \
                        dfmcd['FIPS MCD code'].str.zfill(5)
dfmcd.head()

,Summary level code,FIPS state code,FIPS county code,FIPS MCD code,State name,County name,MCD name,resident pop estimate,resident pop MOE,working in MCD,...,commuting pop change MOE,commuting pop percent,commuting pop percent MOE,live and work,live and work MOE,live and work percent,live and work percent MOE,work residence ratio,work residence ratio MOE,census block
0,60,9,1,4720,Connecticut,Fairfield County,Bethel town,18377,27,7377,...,731,-9.3,4.0,2033,290,22.4,2.7,0.81,0.07,0900104720
1,60,9,1,8070,Connecticut,Fairfield County,Bridgeport town,142576,45,46183,...,1544,-11.2,1.1,19323,1048,31.1,1.6,0.74,0.02,0900108070
2,60,9,1,8980,Connecticut,Fairfield County,Brookfield town,16214,25,6704,...,572,-10.6,3.5,2064,288,24.5,3.3,0.80,0.06,0900108980
3,60,9,1,18500,Connecticut,Fairfield County,Danbury town,79371,40,46553,...,1444,6.7,1.8,18462,1061,44.8,2.2,1.13,0.04,0900118500
4,60,9,1,18850,Connecticut,Fairfield County,Darien town,20414,37,8888,...,714,6.4,3.5,1949,225,25.7,2.5,1.17,0.10,0900118850


In [122]:
# Doesn't have all the places in it... 
pd.merge(df, dfmcd[['MCD name', 'census block']], how='left', left_on=['Census Block'], right_on=['census block'])

,Census Block,Population,MCD,Name,District,County,MCD name,census block
0,3400543740,19131,Y,,001,Burlington,Maple Shade township,3400543740
1,3400555800,7398,Y,,001,Burlington,Palmyra borough,3400555800
2,3400702200,8819,Y,,001,Camden,Audubon borough,3400702200
3,3400702230,1023,Y,,001,Camden,NaN,NaN
4,3400703250,6983,Y,,001,Camden,Barrington borough,3400703250
5,3400704750,11583,Y,,001,Camden,Bellmawr borough,3400704750
6,3400705440,7588,Y,,001,Camden,Berlin borough,3400705440
7,3400705470,5357,Y,,001,Camden,Berlin township,3400705470
8,3400708170,1955,Y,,001,Camden,NaN,NaN
9,3400710000,77344,Y,,001,Camden,Camden city,3400710000


In [123]:
## Try something else... the Gazetteer from http://www.census.gov/geo/maps-data/data/gazetteer2010.html
dfgaz = pd.read_csv('../2017 Other data/Gaz_cousubs_national.csv')
dfgaz['GEOID'] = dfgaz['GEOID'].astype(str)
dfgaz[dfgaz['USPS'] == 'NJ']

,USPS,GEOID,ANSICODE,NAME,FUNCSTAT10,POP10,HU10,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
19726,NJ,3400100000,0,County subdivisions not defined,F,0,0,0,141638182,0.000,54.687,39.377235,-74.362448
19727,NJ,3400100100,885134,Absecon city,F,8411,3365,13975074,4911452,5.396,1.896,39.423563,-74.493025
19728,NJ,3400102080,885142,Atlantic City city,F,39558,20013,27834698,16289908,10.747,6.290,39.377297,-74.451082
19729,NJ,3400107810,885171,Brigantine city,F,9450,9222,16543185,10301305,6.387,3.977,39.427291,-74.389617
19730,NJ,3400108680,885173,Buena borough,F,4603,1855,19626218,3410,7.578,0.001,39.529170,-74.945205
19731,NJ,3400108710,882048,Buena Vista township,A,7570,3008,106328785,1229410,41.054,0.475,39.513086,-74.880028
19732,NJ,3400115160,885192,Corbin City city,F,492,212,19853242,3302576,7.665,1.275,39.301674,-74.705578
19733,NJ,3400120290,882051,Egg Harbor township,A,43323,16347,172487508,21589686,66.598,8.336,39.378291,-74.599779
19734,NJ,3400120350,885204,Egg Harbor City city,F,4243,1736,28314199,1310875,10.932,0.506,39.563687,-74.595865
19735,NJ,3400121870,885212,Estell Manor city,F,1735,673,138099831,4604332,53.321,1.778,39.353933,-74.775135


In [124]:
df = pd.merge(df, dfgaz[['NAME', 'GEOID']], 
              how='left', left_on=['Census Block'], right_on=['GEOID'])
df[df['MCD']=='Y']['NAME'].value_counts()

Washington township                5
Franklin township                  3
Point Pleasant borough             2
Monroe township                    2
Union township                     2
Greenwich township                 2
Mansfield township                 2
Newark city                        2
Kearny town                        2
Teaneck township                   2
Springfield township               2
County subdivisions not defined    2
West Orange township               2
Bayonne city                       2
Jersey City city                   2
Scotch Plains township             2
Stafford township                  2
East Greenwich township            2
Middletown township                2
Old Bridge township                2
Ocean township                     2
Montclair township                 2
Bloomfield township                2
Morris Plains borough              1
Saddle River borough               1
Mantoloking borough                1
Dover town                         1
S

In [131]:
tot = df['Population'].sum()
urbtot = df[df['MCD']=='Y']['Population'].sum()
rurtot = df[df['MCD']=='N']['Population'].sum()
print('NJ total population in 2011 plan components file {} ({} urban, {} rural ({}%))'.format(tot, urbtot, rurtot, 
                                                              100*rurtot/tot))

NJ total population in 2011 plan components file 9134693 (8068750 urban, 1065943 rural (11.669171585733642%))


In [133]:
df11 = df[df['District'] == '011']
tot = df11['Population'].sum()
urbtot = df11[df11['MCD']=='Y']['Population'].sum()
rurtot = df11[df11['MCD']=='N']['Population'].sum()
print('NJ total population in 2011 plan components file {} ({} urban, {} rural ({}%))'.format(tot, urbtot, rurtot, 
                                                              100*rurtot/tot))

NJ total population in 2011 plan components file 794877 (732658 urban, 62219 rural (7.827500355400899%))
